In [1]:
import pandas as pd
import numpy as np
from simpletransformers.classification import ClassificationModel,ClassificationArgs
import torch

C:\Users\a2285\anaconda3\envs\st\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df=pd.read_csv('sum1.csv',header=None)

C:\Users\a2285\AppData\Local\Temp\ipykernel_21392\266863027.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('sum1.csv',header=None)


In [3]:
df

,0,1,2,3,4,5,6,7
0,tt0000001,The First Movie In A Popular Genre\n,Snow Leopard,31 August 2005,23,24,NaN,This is the first movie in what quickly became...
1,tt0000001,There are two versions of this movie.\n,F Gwynplaine MacIntyre,1 February 2008,14,15,7.0,Here's a perfect example of the pitfalls of wr...
2,tt0000001,Some serious twirling and kicking involved.\n,Boba_Fett1138,25 November 2007,9,10,6.0,This film is part of the series of short Ediso...
3,tt0000001,Edison: Invention of Movies\n,Michael_Elliott,31 December 2008,4,4,NaN,Carmencita (1894) *** 1/2 (out of 4)Infamous E...
4,tt0000001,one-shot record of a belly dancer\n,Brian-59,12 August 1998,26,30,NaN,"""Carmencita Dancing,"" one of a series of Ediso..."
...,...,...,...,...,...,...,...,...
938759,tt9914458,Theoretical\n,knightx-1,6 April 2019,8,19,5.0,"Many people would base their ""expert"" opinion ..."
938760,tt9914458,Not Biblical truth from what I've read...\n,jesusistheiam,12 June 2019,0,2,1.0,Read the Bible over a period of months from be...
938761,tt9914644,Very well done\n,koachkatieaz,7 November 2021,2,2,10.0,I've watched many many docs about this terribl...
938762,tt9914644,Inside.........\n,hiltonsmithjr,21 October 2019,3,9,9.0,How do you discount 2 guys bringing bringing d...


### 將原本1~10分 轉為同NLP訓練的三分類 先忽略NaN

In [4]:
# 將類別轉換為三元
def convert_rating_to_category(rating):
    if pd.isnull(rating):
        return rating
    elif 1 <= rating <= 5:
        return 0
    elif 6 <= rating <= 8:
        return 1
    elif 9 <= rating <= 10:
        return 2


# 對第6欄的數值應用函數 convert_rating_to_category 進行轉換
df[6] = df[6].apply(convert_rating_to_category)


### 呼叫訓練好的模型，並將NaN利用模型預測後再補回原本的資料

In [5]:
# 呼叫模型

output_dir = "C:/Users/a2285/workspace/電影專題/outputs/roberta-base-bs-48-ep-10-cls-model 分3類/checkpoint-34000"

model_args = ClassificationArgs()
model_args.num_labels = 3
model_args.use_multiprocessing = True
model_args.use_multiprocessing_for_evaluation = True
model_args.eval_batch_size = 128  # 更改批量大小
use_multiprocessing_for_evaluation=True
# model_args.fp16 = True  # 啟用半精度

loaded_model = ClassificationModel(
    model_type="roberta",
    model_name=output_dir,
    use_cuda=torch.cuda.is_available(),
    cuda_device=0,
    num_labels=3,
    args=model_args,
)


In [6]:
df[6].isna().sum()

120100

In [7]:
import itertools

def predict_in_batches(texts, model, batch_size=128):
    return list(itertools.chain.from_iterable(
        model.predict(texts[i:i + batch_size])[0]
        for i in range(0, len(texts), batch_size)
    ))

# 找到第6欄是 NaN 的所有行，並對其進行預測
df.loc[df[6].isna(), 6] = predict_in_batches(df[df[6].isna()][7].tolist(), loaded_model)



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.32it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.18it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.27it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.88it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.21it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.15it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.84it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.76it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.94it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.13it/s]


In [11]:
df[6].value_counts()

1.0    394684
0.0    302326
2.0    241754
Name: 6, dtype: int64

In [12]:
df.to_csv('review_3type',index=False)